<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2024notebooks/2021_0618pnt_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PNT 画像を使ってディープラーニングモデルによる転移学習を行う PyTorch デモ
- author: 浅川伸一
- date: 2021-0618
- filename: 2021_0618pnt_transfer_learning.ipynb

In [ ]:
%config InlineBackend.figure_format = 'retina'
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

from IPython import get_ipython
isColab =  'google.colab' in str(get_ipython())
if isColab: #ライブラリのインストール

    #`import bit` する前に termcolor を downgrade しないと colab ではテキストに色がつかない\
    !pip install --upgrade termcolor==1.1
    #import termcolor

    !git clone https://github.com/project-ccap/ccap.git

    #ImageNet のサンプルデータをダウンロード
    !wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1xKXbovkEQwdJefzCuaS_a351LUIuRz-1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1xKXbovkEQwdJefzCuaS_a351LUIuRz-1" -O ccap_data.tgz && rm -rf /tmp/cookies.txt
    !tar xzf ccap_data.tgz

# 各画像の画面表示時に日本語キャプションを付与する準備
import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize-matplotlib
    import japanize_matplotlib

#  ImageNet の各ラベルの WordNet ID 処理用
import nltk
nltk.download('wordnet')
if isColab:
    nltk.download('omw-1.4')
else:
    nltk.download('omw')

import numpy as np
import PIL

In [ ]:
# 以下は動作確認，ImageNet の利用
# ただし本来 ImageNet の画像利用には登録が必要である
# そのため，利用時には各ユーザの責任において ImageNet への登録申請を行うこと
# 参照 URL: http://image-net.org/download-images
# 文献: J. Deng, W. Dong, R. Socher, L.-J. Li, K. Li and L. Fei-Fei, ImageNet: A Large-Scale Hierarchical Image Database,
#       IEEE Computer Vision and Pattern Recognition (CVPR), 2009.
from ccap import imagenetDataset
imagenet = imagenetDataset()
# img_no = int(input('0から999までの数字を入力して下さい'))
# imagenet.sample_and_show(img_no)

nrows, ncols = 6, 4
#ncols = 5
fig, fig_axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols * 3.4, nrows * 2.4), constrained_layout=True)
#fig, fig_axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols * 1.4, nrows * 1.4), constrained_layout=True)
# constrained_layout は subplot や 凡例やカラーバーなどの装飾を自動的に調整して，
# ユーザが要求する論理的なレイアウトをできるだけ維持しながら， 図ウィンドウに収まるようにします。

Ns = np.random.permutation(1000)
for i in range(nrows):
    for j in range(ncols):
        _x = i * 10 + j
        x = int(Ns[_x])
        img = PIL.Image.open(imagenet.sample_image(x)).convert('RGB')
        fig_axes[i][j].imshow(img)
        fig_axes[i][j].axis('off')
        label = imagenet.data[x]['label_ja'][0] if len(imagenet.data[x]['label_ja']) > 0 else imagenet.data[x]['label'][0]
        fig_axes[i][j].set_title(f'{x} {label}') #imagenet.data[x]["label"][0]}')


In [ ]:
from ccap import pntDataset
pnt = pntDataset()
pnt.show_all_images()  # 全データ画像表示時間がかかる

# ここから先は PyTorch を使った転移学習の実際

In [ ]:
import numpy as np
import PIL.Image as PILImage
from scipy.special import logsumexp, softmax
from termcolor import colored

import torch
import torchvision
from torchvision import transforms
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim

import torchvision.models as models
#コメントアウトしてある，他のモデルを試すことも可能
resnet18 = models.resnet18(weights="DEFAULT", progress=True)
#alexnet = models.alexnet()
#vgg16 = models.vgg16()
#squeezenet = models.squeezenet1_0()
#densenet = models.densenet161()
#inception = models.inception_v3()
#googlenet = models.googlenet()
#shufflenet = models.shufflenet_v2_x1_0()
#mobilenet = models.mobilenet_v2()
#resnext50_32x4d = models.resnext50_32x4d()
#wide_resnet50_2 = models.wide_resnet50_2()
#mnasnet = models.mnasnet1_0()

#net = models.mnasnet1_0(pretrained=True, progress=True)

import copy
net = copy.deepcopy(resnet18)

In [ ]:
pnt_img_path = [pnt.data[k]['img'] for k in pnt.data.keys()]
pnt_name_dict = {i:k for i, k in enumerate(pnt.data.keys())}
#print(pnt_name_dict)
#pnt_img_path

In [ ]:
# 入力画像の前処理をするクラス。訓練時と推論時で処理が異なる
class ImageTransform():
    """
    画像の前処理クラス。訓練時、検証時で異なる動作をする。
    画像のサイズをリサイズし、色を標準化する。
    訓練時は RandomResizedCrop と RandomHorizontalFlip で データ拡張

    Attributes
    ----------
    resize : int
        リサイズ先の画像の大きさ。
    mean : (R, G, B)
        各色チャネルの平均値。
    std : (R, G, B)
        各色チャネルの標準偏差。
    """
    def __init__(self,
                 resize:int=(224),
                 mean:tuple=(0.485, 0.456, 0.406),
                 std:tuple=(0.229, 0.224, 0.225)):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(
                    size=resize,
                    #scale=(0.95, 1.0)),              # データ拡張
                    scale=(0.8, 1.0)),              # データ拡張
                #transforms.RandomHorizontalFlip(),  # フリップしてしまうと時計の文字盤が反転してしまうから，反転させない
                transforms.RandomAffine(
                    #degrees=(-5,5),
                    degrees=(-10,10),
                    #degrees=(-20,20),
                    translate=None,
                    #scale=[0.9,1.1]
                    scale=[0.7,1.0],
                    fill=255,
                ),
                transforms.ToTensor(),          # テンソルに変換
                transforms.Normalize(mean, std) # 標準化
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),      # リサイズ
                transforms.CenterCrop(resize),  # 画像中央を resize×resize で切り取り
                transforms.ToTensor(),          # テンソルに変換
                transforms.Normalize(mean, std) # 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        """
        Parameters
        ----------
        phase : 'train' or 'val'
            前処理のモードを指定。
        """
        return self.data_transform[phase](img)

In [ ]:
# Dataset の作成
class pnt_torch_Dataset(data.Dataset):
    """
    PNT のDatasetクラス

    Attributes
    ----------
    file_list : list
        画像のパスを格納したリスト
    transform : object
        前処理クラスのインスタンス
    phase : str
        'train': 訓練時
        'test': 検証時
    """

    def __init__(self,
                 file_list:list=pnt_img_path,
                 name_dict:dict=pnt_name_dict,
                 transform=ImageTransform,
                 phase='train'):
        super().__init__()
        self.file_list = file_list  # ファイルパスのリスト
        self.transform = transform # Imagetransform  # 前処理クラスのインスタンス
        #self.transform = naive_transform  # 前処理クラスのインスタンス
        self.phase = phase  # train or valの指定
        self.namedict = name_dict

    def __len__(self):
        '''画像の枚数を返す'''
        return len(self.file_list)

    def __getitem__(self, idx):
        '''前処理をした画像の Tensor 形式のデータとラベルを取得'''

        # idx 番目の画像を取得
        img_path = self.file_list[idx]
        img = PILImage.open(img_path)

        # 画像の前処理を実施
        img_transformed = self.transform(img)

        # ファイル名から画像のラベルを抜出し
        label = self.namedict[idx]
        return img_transformed, label



# 画像の前処理に必要なパラメータの定義
# size = 224
# mean = (0.485, 0.456, 0.406)
# std = (0.229, 0.224, 0.225)

train_dataset = pnt_torch_Dataset(file_list=pnt_img_path,
                                  name_dict=pnt_name_dict,
                                  #transform=ImageTransform_broken(),
                                  transform=ImageTransform(),
                                  phase='train')

val_dataset = pnt_torch_Dataset(file_list=pnt_img_path,
                                name_dict=pnt_name_dict,
                                transform=ImageTransform(),
                                phase='val')

# 動作確認
idx = 3
print(train_dataset.__getitem__(idx)[0].size())
print(train_dataset.__getitem__(idx)[1])
print(train_dataset.__len__())

In [ ]:
# ミニバッチのサイズの設定
#batch_size = 32
batch_size = 48

# DataLoaderを作成
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数へまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

# 動作確認
batch_iterator = iter(dataloaders_dict["train"])  # イテレータに変換
inputs, labels = next(batch_iterator)  # 1番目の要素を取り出す
print(inputs.size())
print(labels)

In [ ]:
# 事前学習済のモデル構成を表示
net.eval();
for k, v in net.named_parameters():
    print(k, v.data.size(), v.requires_grad)

In [ ]:
# 直上出力の最後 `Linear(in_features=1280, out_features=1000, bias=True)` に注目
# モデルの最終直下層の出力ユニット数を pnt に合わせて 184 にする
#net.classifier[1] = nn.Linear(in_features=1280, out_features=184)
net.fc = nn.Linear(in_features=512, out_features=184)
net.eval();

In [ ]:
# 転移学習で学習させるパラメータを params_to_update に格納
params_to_update = []

# 学習させるパラメータ名
update_param_names = ["classifier.1.weight", "classifier.1.bias"]
update_param_names = ["fc.weight", "fc.bias"]

# 学習させるパラメータ以外は勾配計算をなくし、変化しないように設定
for name, param in net.named_parameters():
    if name in update_param_names:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

# params_to_update を表示
#print(type(params_to_update[0]))
#print(dir(params_to_update[0]))
#print(dir(params_to_update[1]))

In [ ]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()

# 最適化手法の設定
#optimizer = optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)
optimizer = optim.Adam(params=params_to_update, lr=0.01)

In [ ]:
# モデルを学習させる関数
def train_model(net:torch.nn.Module=None,
                dataloaders_dict:dict=None,
                criterion:torch.nn.modules.loss=None,
                optimizer:torch.optim=None,
                num_epochs:int=0):

    # epochのループ
    for epoch in range(num_epochs):
        print(f'エポック {epoch+1}/{num_epochs}', end="\t")
        #print('Epoch {}/{}'.format(epoch+1, num_epochs))
        #print('-------------')

        # epochごとの学習と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モード
            else:
                net.eval()   # モデルを検証モード

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # 未学習時の検証性能を確かめるため epoch=0 の訓練は省略
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in dataloaders_dict[phase]:
                optimizer.zero_grad() # optimizerを初期化

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # loss の合計を更新
                    epoch_loss += loss.item() * inputs.size(0)
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epoch ごとの損失と正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            if phase == 'train':
                phase_ja = '訓練'
                end="\t"
            else:
                phase_ja = '検証'
                end="\n"
            print(f'{phase_ja}損失: {epoch_loss:.3f} 精度: {epoch_acc:.3f}', end=end)
            #print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            #    phase, epoch_loss, epoch_acc))

In [ ]:
%%time
# 学習・検証の実行
num_epochs=10
#num_epochs=20
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


In [ ]:
# #訓練したデータを保存
# saved_weight_file = '2024_0706pnt_resnet18.pth'
# torch.save(net.state_dict(), saved_weight_file)
# load_weights = torch.load(saved_weight_file)
# net.load_state_dict(load_weights)

In [ ]:
# #pnt_name_dict
# pnt_labels = [fname[19:].replace('.png','') for fname in pnt_img_path]
# pnt_label2idx = {label:pnt_labels.index(label) for label in pnt_labels}
# print(pnt_label2idx)
# print(pnt_labels)

In [ ]:
#float_formatter = "{:.3f}".format
#np.set_printoptions(formatter={'float_kind':float_formatter})
# see https://note.nkmk.me/python-numpy-set-printoptions-float-formatter/
np.set_printoptions(formatter={'int': '{:3d}'.format, 'float_kind':'{:.3f}'.format})

def diagnose(no,
             name_dict=pnt_name_dict,
             #num2word=_num2word,
             #id2word=_id2word,
             #filelist=_img_file_list,
             filelist=pnt_img_path,
             display=False,
             n_best=5):
    _id = name_dict[no]
    img_file = filelist[no]
    img = PILImage.open(img_file)

    #label = num2word[no]
    label = pnt_labels[no]

    # 画像の前処理と処理済み画像の表示
    size = 224
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)

    transform = ImageTransform(size, mean, std)
    img_transformed = transform(img, phase="val")  # torch.Size([3, 224, 224])

    # (色、高さ、幅)を (高さ、幅、色)に変換し、0-1に値を制限して表示
    if display:
        img_transformed_ = img_transformed.numpy().transpose((1, 2, 0))
        img_transformed_ = np.clip(img_transformed_, 0, 1)
        plt.axis(False); plt.imshow(img_transformed_);plt.show()

    # 認識の実施
    inputs = transform(img, phase='val')
    inputs_ = inputs.unsqueeze_(0)
    out = net(inputs_)
    outnp = out.detach().numpy()
    ids = np.argsort( - outnp[0])
    sftmx = softmax(-outnp[0])

    print(no, end=" ")
    OK = True
    if _id == ids[0]:
        print('Hit ', end="")
    else:
        print(colored('Miss', 'red'), end="")
        OK = False

    print(ids[:n_best], end=" ")
    for id_ in ids[:n_best]:
        #print(id2word[id_], end=" ")
        print(pnt_labels[id_], end=" ")
    print(- np.sort(-sftmx)[:n_best])
    if not OK:
        img_transformed_ = img_transformed.numpy().transpose((1, 2, 0))
        img_transformed_ = np.clip(img_transformed_, 0, 1)
        #plt.title('正解:{0}, 出力:{1},{2},{3}'.format(vocabs_i2w[id],
        #                                          vocabs_i2w[ids[0]],
        #                                          vocabs_i2w[ids[1]],
        #                                          vocabs_i2w[ids[2]]))
        plt.title('正解:{0}, 出力:{1},{2},{3}'.format(label,
                                                  pnt_labels[ids[0]],
                                                  pnt_labels[ids[1]],
                                                  pnt_labels[ids[2]],))
        #plt.title('正解:{0}, 出力:{1},{2},{3}'.format(num2word[no],
        #                                          id2word[ids[0]],
        #                                          id2word[ids[1]],
        #                                          id2word[ids[2]]))
        plt.axis(False);plt.imshow(img_transformed_)
        plt.show()


#for i in range(len(_img_file_list)): # tlpa_sala)):
#for i in [3,5,10]:
for i in range(len(pnt_name_dict)):
    diagnose(i, display=False, n_best=5)